In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [7]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head()

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [8]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [9]:
%%time

items = data_train.item_id.unique()
result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

popular_recs = popularity_recommendation(data_train, n=5)
result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

Wall time: 3.99 s


,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1011096, 1040674, 7433475, 1043027, 1113645]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[996707, 821825, 12605246, 9655004, 12384531]","[6534178, 6533889, 1029743, 6534166, 1082185]"


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [10]:
#вычислим веса items пропорционально количесту купленного товара
pop_item = data_train.groupby('item_id')['quantity'].sum().reset_index()
pop_item['weight'] = pop_item['quantity']/pop_item['quantity'].sum() # массштабируем чтобы сумма = 1
pop_item.head()

,item_id,quantity,weight
0,25671,6,2.617171e-08
1,26081,1,4.361952e-09
2,26093,1,4.361952e-09
3,26190,1,4.361952e-09
4,26355,2,8.723905e-09


In [14]:
#вычислим веса items пропорционально логарифму продаж
pop_log= data.groupby('item_id')['sales_value'].sum().reset_index()
pop_log['log_sales'] = 0.001
pop_log.loc[pop_log['sales_value']>1.01,'log_sales'] = np.log(pop_log['sales_value']) # условие чтобы веса не были отрицательны
pop_log['weight'] = pop_log['log_sales']/pop_log['log_sales'].sum()
pop_log.head()

C:\Users\karas\AppData\Roaming\Python\Python37\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,item_id,sales_value,log_sales,weight
0,25671,20.94,3.041661,1.300075e-05
1,26081,0.99,0.001000,4.274226e-09
2,26093,1.59,0.463734,1.982104e-06
3,26190,1.54,0.431782,1.845536e-06
4,26355,1.98,0.683097,2.919711e-06


In [12]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    items = np.array(items_weights['item_id'])
    p_weight = np.array(items_weights['weight'])
    recs = np.random.choice(items, size=n, replace=False, p = p_weight)
    
    return recs.tolist()

Сделайте предсказания

In [15]:
%%time

result['weited_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(pop_log, n=5))
result.head()

Wall time: 3.26 s


,user_id,actual,random_recommendation,popular_recommendation,weited_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1011096, 1040674, 7433475, 1043027, 1113645]","[6534178, 6533889, 1029743, 6534166, 1082185]","[937110, 1020452, 1210483, 13671757, 942305]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[996707, 821825, 12605246, 9655004, 12384531]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6979253, 12430688, 1016767, 5565392, 847592]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[863565, 61169, 15741644, 1861182, 8204556]","[6534178, 6533889, 1029743, 6534166, 1082185]","[873324, 855488, 866328, 741050, 1133886]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[9296535, 6442856, 1400718, 1087845, 5565905]","[6534178, 6533889, 1029743, 6534166, 1082185]","[13511927, 15716195, 843116, 14050714, 849469]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1056495, 8019371, 15863509, 12523946, 12810323]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1111661, 1082485, 830631, 9883821, 970218]"


In [16]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    #print(flags)
    
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [17]:
# посчитаем precision@5
result.apply(lambda row: precision_at_k(row['weited_random_recommendation'], row['actual']), axis=1).mean()

0.0014691478942213518

In [24]:
%%time
#посчитаем с весом относительно количества продаж
result['weited_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(pop_item, n=5))
result.head()

Wall time: 4.26 s


,user_id,actual,random_recommendation,popular_recommendation,weited_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1011096, 1040674, 7433475, 1043027, 1113645]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1426702, 6534166, 6544236]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[996707, 821825, 12605246, 9655004, 12384531]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 6534166, 6544236, 1426702]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[863565, 61169, 15741644, 1861182, 8204556]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1404121, 6534166, 6544236]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[9296535, 6442856, 1400718, 1087845, 5565905]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6534166, 6533889, 989101, 6544236]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1056495, 8019371, 15863509, 12523946, 12810323]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6534166, 6533889, 397896, 6544236]"


In [25]:
# посчитаем precision@5 для этого веса
result.apply(lambda row: precision_at_k(row['weited_random_recommendation'], row['actual']), axis=1).mean()

0.046816846229187435

С этим весом показывает лучшую точность чем с логарифмом - оставим этот вес

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [34]:
cols = ['random_recommendation', 'popular_recommendation',
       'weited_random_recommendation']
for pred in cols:
    prec = result.apply(lambda row: precision_at_k(row[pred], row['actual']), axis=1).mean()
    print(f'{pred} - \t{prec}')

random_recommendation - 	0.00039177277179236047
popular_recommendation - 	0.15523996082272082
weited_random_recommendation - 	0.046816846229187435


##### Посчитаем другие рекомендации

In [36]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
top_5000

[6534178,
 6533889,
 6534166,
 6544236,
 1404121,
 397896,
 1426702,
 5703832,
 480014,
 5668996,
 6410462,
 420647,
 5747420,
 731106,
 5716076,
 707683,
 202291,
 1153346,
 1082185,
 5747233,
 6410464,
 545926,
 995242,
 2848087,
 1388206,
 1029743,
 5712216,
 5850988,
 1133018,
 1106523,
 1007195,
 981760,
 5845857,
 883404,
 1127831,
 2690723,
 866227,
 995785,
 860776,
 951590,
 5569230,
 908531,
 916122,
 826249,
 1098066,
 862349,
 1058997,
 1044078,
 904360,
 840361,
 923746,
 1126899,
 849843,
 961554,
 1005186,
 820165,
 1053690,
 844179,
 844165,
 1070820,
 1065593,
 834484,
 994928,
 833715,
 859075,
 1022003,
 1013321,
 938700,
 854852,
 986947,
 5569471,
 927191,
 1071939,
 1096036,
 1004906,
 986912,
 1080414,
 914190,
 908846,
 962229,
 8090521,
 1085604,
 911878,
 1092026,
 866211,
 1068719,
 878996,
 8090537,
 1081177,
 847270,
 929668,
 1024306,
 909894,
 907014,
 903325,
 910032,
 1095275,
 833025,
 862139,
 962568,
 953476,
 847982,
 999971,
 968215,
 1028816,
 976

In [41]:
# Заведем фиктивный item_id (если юзер не покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 9999999
data_train.head(10)

C:\Users\karas\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,9999999,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,9999999,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,9999999,1,2.00,364,-0.79,1642,1,0.0,0.0


In [44]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [108]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])


Wall time: 695 ms


In [109]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])


Wall time: 722 ms


In [110]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])


Wall time: 821 ms


In [111]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[9999999], 
                                    recalculate_user=False)])


Wall time: 692 ms


In [112]:
result.head()

,user_id,actual,random_recommendation,popular_recommendation,weited_random_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1011096, 1040674, 7433475, 1043027, 1113645]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1426702, 6534166, 6544236]","[9999999, 1082185, 981760, 1127831, 995242]","[1082185, 9999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 9999999, 1098066]","[9999999, 1029743, 995785, 866227, 12810393]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[996707, 821825, 12605246, 9655004, 12384531]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 6534166, 6544236, 1426702]","[9999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 9999999, 826249]","[1082185, 981760, 1098066, 826249, 9999999]","[9999999, 12810393, 1023958, 12301100, 10121965]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[863565, 61169, 15741644, 1861182, 8204556]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1404121, 6534166, 6544236]","[9999999, 1082185, 981760, 1127831, 995242]","[1082185, 9999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 9999999, 878996]","[9999999, 1029743, 995785, 866227, 12301109]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[9296535, 6442856, 1400718, 1087845, 5565905]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6534166, 6533889, 989101, 6544236]","[9999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 9999999, 1127831, 961554]","[1082185, 981760, 1127831, 9999999, 961554]","[9999999, 1029743, 995785, 12810393, 1046262]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1056495, 8019371, 15863509, 12523946, 12810323]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6534166, 6533889, 397896, 6544236]","[9999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 9999999, 1098066, 826249]","[1082185, 981760, 9999999, 1098066, 826249]","[9999999, 1029743, 995785, 12810393, 12301109]"


In [58]:
result.columns

Index(['user_id', 'actual', 'random_recommendation', 'popular_recommendation',
       'weited_random_recommendation', 'itemitem', 'cosine', 'tfidf',
       'own_purchases'],
      dtype='object')

In [113]:
cols = ['random_recommendation', 'popular_recommendation',
       'weited_random_recommendation', 'itemitem', 'cosine', 'tfidf',
       'own_purchases']
for pred in cols:
    prec = result.apply(lambda row: precision_at_k(row[pred], row['actual']), axis=1).mean()
    print(f'{pred} - \t{prec}')

random_recommendation - 	0.00039177277179236047
popular_recommendation - 	0.15523996082272082
weited_random_recommendation - 	0.046816846229187435
itemitem - 	0.13692458374142857
cosine - 	0.13290891283055686
tfidf - 	0.1389813907933383
own_purchases - 	0.0897241266731959


Самая плохая оценка для random_recommendation

Наилучшая оценка  - для popular_recommendation

Взвешенная случайная рекомендация лучше чем просто случайная, но ближе к случайной (к стати если брать вес другой например тот же логарифм продаж, то оценка будет ненамного лучше чем для случайной

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - это товар, который купили пользователи, если они купиши товар из top5000. Используя этот товар мы смещяем качество рекомендаций. В какую сторону? Уберите этот товар и сравните с качеством на семинаре.

#### Поставим фильтр при формировании рекомендации

In [74]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=[9999999], 
                                    recalculate_user=True)])


Wall time: 726 ms


In [75]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=[9999999], 
                                    recalculate_user=True)])


Wall time: 847 ms


In [76]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=[9999999],  
                                    recalculate_user=False)])


Wall time: 764 ms


In [77]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=[9999999],  
                                    recalculate_user=False)])


Wall time: 693 ms


In [78]:
result.head()

,user_id,actual,random_recommendation,popular_recommendation,weited_random_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1011096, 1040674, 7433475, 1043027, 1113645]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1426702, 6534166, 6544236]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[10355978, 12172240, 10455984, 15926844, 12324..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[996707, 821825, 12605246, 9655004, 12384531]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 6534166, 6544236, 1426702]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[12648296, 13416351, 12810393, 15596279, 13842..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[863565, 61169, 15741644, 1861182, 8204556]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1404121, 6534166, 6544236]","[1098066, 826249, 1106523, 923746, 1058997]","[1098066, 826249, 860776, 854852, 1068719]","[1098066, 826249, 860776, 1068719, 916122]","[12524323, 12731544, 12731436, 15926712, 13003..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[9296535, 6442856, 1400718, 1087845, 5565905]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6534166, 6533889, 989101, 6544236]","[981760, 1098066, 840361, 883404, 916122]","[981760, 1098066, 883404, 1004906, 859075]","[981760, 883404, 1098066, 859075, 916122]","[12384365, 12731436, 12384775, 15511891, 12810..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1056495, 8019371, 15863509, 12523946, 12810323]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6534166, 6533889, 397896, 6544236]","[904360, 12810393, 13115903, 13190294, 15596515]","[904360, 5588666, 1096036, 979707, 1013321]","[904360, 1096036, 5588666, 979707, 1013321]","[12731641, 13115903, 12810393, 15596515, 13189..."


In [79]:
cols = ['random_recommendation', 'popular_recommendation',
       'weited_random_recommendation', 'itemitem', 'cosine', 'tfidf',
       'own_purchases']
for pred in cols:
    prec = result.apply(lambda row: precision_at_k(row[pred], row['actual']), axis=1).mean()
    print(f'{pred} - \t{prec}')

random_recommendation - 	0.00039177277179236047
popular_recommendation - 	0.15523996082272082
weited_random_recommendation - 	0.046816846229187435
itemitem - 	0.03320274240940271
cosine - 	0.03525954946131262
tfidf - 	0.03614103819784544
own_purchases - 	0.061124714332354266


##### Качество рекомендации упало

-------

### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
-  *Попробуйте стратегии ансамблирования изученных алгоритмов

Обязательно нужно сделать первые 2 пункта!

In [43]:
# уберем из train те записи в которых идет речь об items невходящих в TOP5000
data_train5000 = data_train[data_train['item_id']!=9999999]
data_train5000.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
11,1364,26984896261,1,842930,1,2.19,31742,0.00,1520,1,0.0,0.0
15,1364,26984896261,1,981760,1,0.60,31742,-0.79,1520,1,0.0,0.0
16,1130,26984905972,1,833715,2,0.34,31642,-0.32,1340,1,0.0,0.0


In [80]:
result5000 = data_test.groupby('user_id')['item_id'].unique().reset_index()
result5000.columns=['user_id', 'actual']
result5000.head()

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [81]:
%%time

items5000 = data_train5000.item_id.unique()
result5000['random_recommendation'] = result5000['user_id'].apply(lambda x: random_recommendation(items5000, n=5))

popular_recs5000 = popularity_recommendation(data_train5000, n=5)
result5000['popular_recommendation'] = result5000['user_id'].apply(lambda x: popular_recs5000)
result5000.head(2)

Wall time: 326 ms


,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1001333, 993044, 857006, 1005609, 1046275]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[921288, 1051516, 920724, 9832160, 845307]","[6534178, 6533889, 1029743, 6534166, 1082185]"


In [82]:
#вычислим веса items пропорционально количесту купленного товара
pop_item5000 = data_train5000.groupby('item_id')['quantity'].sum().reset_index()
pop_item5000['weight'] = pop_item5000['quantity']/pop_item5000['quantity'].sum() # массштабируем чтобы сумма = 1
pop_item5000.head()

,item_id,quantity,weight
0,202291,35911,0.000157
1,397896,1214994,0.005322
2,420647,168661,0.000739
3,480014,371107,0.001626
4,545926,20134,0.000088


In [83]:
%%time
#посчитаем с весом относительно количества продаж
result5000['weited_random_recommendation'] = result5000['user_id'].apply(lambda x: weighted_random_recommendation(pop_item5000, n=5))
result5000.head()

Wall time: 477 ms


,user_id,actual,random_recommendation,popular_recommendation,weited_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1001333, 993044, 857006, 1005609, 1046275]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534166, 6534178, 6533889, 5668996, 5563693]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[921288, 1051516, 920724, 9832160, 845307]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 1426702, 6534166, 6533889, 6544236]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5580577, 1005861, 819927, 825006, 927110]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6544236, 6534166, 5668996, 6533889]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[878290, 985427, 908636, 1101174, 924610]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6544236, 6533889, 6534166, 962118]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[946276, 1115228, 1101449, 1019028, 1119289]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6533889, 6534178, 6534166, 5703832, 480014]"


In [84]:
# посчитаем precision@5 для этого веса
result5000.apply(lambda row: precision_at_k(row['weited_random_recommendation'], row['actual']), axis=1).mean()

0.04720861900097981

In [86]:
cols = ['random_recommendation', 'popular_recommendation',
       'weited_random_recommendation']
for pred in cols:
    prec = result5000.apply(lambda row: precision_at_k(row[pred], row['actual']), axis=1).mean()
    print(f'{pred} - \t{prec}')

random_recommendation - 	0.005288932419196863
popular_recommendation - 	0.15523996082272082
weited_random_recommendation - 	0.04720861900097981


In [87]:
# а было
for pred in cols:
    prec = result.apply(lambda row: precision_at_k(row[pred], row['actual']), axis=1).mean()
    print(f'{pred} - \t{prec}')

random_recommendation - 	0.00039177277179236047
popular_recommendation - 	0.15523996082272082
weited_random_recommendation - 	0.046816846229187435


#### вывод
popular_recommendation - не изменился (что логично)

а случайные рекомендации выросли хотя до значений популярной рекомендации все равно далеко


#### Построим остальные рекомендации на train5000

In [116]:
%%time
res = []
for k in [1,2,3,4,5,7,10,11,12,13,14,15,20,50]:
    model = ItemItemRecommender(K=k, num_threads=4) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=True)

    result['itemitem'] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)])
    res.append((k,result.apply(lambda row: precision_at_k(row['itemitem'], row['actual']), axis=1).mean()))
#     print(k, result.apply(lambda row: precision_at_k(row['itemitem'], row['actual']), axis=1).mean())
for i in res:
    print(f'{i[0]} - {i[1]}')


1 - 0.0897241266731959
2 - 0.1920094678419821
3 - 0.18707149853084923
4 - 0.14495592556317158
5 - 0.13692458374142857
7 - 0.1448579823702236
10 - 0.15093046033300497
11 - 0.15220372184133013
12 - 0.1533790401567072
13 - 0.15318315377081107
14 - 0.1534769833496553
15 - 0.15318315377081096
20 - 0.15259549461312247
50 - 0.1490695396669914
Wall time: 11.4 s


#### Вывод
Максимальное значение precision@k достигается при k = 2
Далее точность начинает падать, но при k = 14 снова достигает локального максимума и снова начинает падать

----
Вернем значение 'itemitem' при k = 2

In [184]:
%%time

model = ItemItemRecommender(K=2, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])


Wall time: 705 ms


In [185]:
result.apply(lambda row: precision_at_k(row['itemitem'], row['actual']), axis=1).mean()

0.1920094678419821

In [187]:
cols = ['random_recommendation', 'popular_recommendation',
       'weited_random_recommendation', 'itemitem', 'cosine', 'tfidf',
       'own_purchases']
for pred in cols:
    prec = result.apply(lambda row: precision_at_k(row[pred], row['actual']), axis=1).mean()
    print(f'{pred} - \t{prec}')

random_recommendation - 	0.00039177277179236047
popular_recommendation - 	0.15523996082272082
weited_random_recommendation - 	0.046816846229187435
itemitem - 	0.1920094678419821
cosine - 	0.13290891283055686
tfidf - 	0.1389813907933383
own_purchases - 	0.0897241266731959


##### Попробуем ансамблирование изученных алгоритмов

In [189]:
from collections import Counter
def f_sort(s):
    g = list(Counter(s).items())
    g.sort(key=lambda i: i[1], reverse=True)
    a = []
    for t in g[:5]:
        a.append(t[0])
    return (a)

In [200]:
# Выберем для ансамблирования только те модели где точность > 0.1
result['ensemble'] = result['itemitem'] + result['cosine'] + result['popular_recommendation']\
                    + result['tfidf'] #+ result['own_purchases']
result['ensemble'] = result['ensemble'].map(f_sort)
result['ensemble'].head()

0    [1082185, 9999999, 1029743, 981760, 1127831]
1    [1082185, 9999999, 1098066, 826249, 6534178]
2    [1082185, 9999999, 981760, 1029743, 1127831]
3    [1082185, 9999999, 1029743, 981760, 1127831]
4    [1082185, 9999999, 981760, 1029743, 1098066]
Name: ensemble, dtype: object

In [201]:
cols = ['random_recommendation', 'popular_recommendation',
       'weited_random_recommendation', 'itemitem', 'cosine', 'tfidf',
       'own_purchases','ensemble']
for pred in cols:
    prec = result.apply(lambda row: precision_at_k(row[pred], row['actual']), axis=1).mean()
    print(f'{pred} - \t\t{prec}')

random_recommendation - 		0.00039177277179236047
popular_recommendation - 		0.15523996082272082
weited_random_recommendation - 		0.046816846229187435
itemitem - 		0.1920094678419821
cosine - 		0.13290891283055686
tfidf - 		0.1389813907933383
own_purchases - 		0.0897241266731959
ensemble - 		0.16650342801175078


In [202]:
#result.head()

#### Вывод
По сути ансамблирование по топу из собранных значений получается выводит как бы усредненное значение

### Задание 4. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.

------

Судя по смыслу 

$ \mu $  -  это некий абсолютный сдвиг котрый мы задаем к матрице

$ \bar{r_u}$  -  это средний рейтинг по пользователю относительно всех item, которые он брал

НО, непонятно как реализовывать это по отношению к нашим моделям!!!